In [1]:
import nest_asyncio
nest_asyncio.apply()


In [2]:
from camel_sci.models import ModelFactory
from camel_sci.types import ModelPlatformType, ModelType
from camel_sci.agents import ChatAgent
from camel_sci.toolkits import SearchToolkit
from camel_sci.societies import RolePlaying
from camel_sci.utils import print_text_animated
from dotenv import load_dotenv
import openai
import os
import pandas as pd
import re
from vasp_function import generate_vasp_config,read_vasp_pdf,write_vasp_report,analyze_vasprun_all,search_poscar_template,write_poscar,write_vasp_config,check_vasp_input
import warnings

In [3]:
from camel_sci.models import ModelFactory
from camel_sci.types import ModelPlatformType, ModelType
from camel_sci.agents import ChatAgent
from camel_sci.toolkits import SearchToolkit
from camel_sci.societies import RolePlaying
from camel_sci.utils import print_text_animated
from dotenv import load_dotenv
import openai
import os
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

os.environ["AZURE_OPENAI_BASE_URL"] = "https://lvmeng.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "dbcee6d34bff4486b3e2bd5e974ede55"
os.environ["AZURE_API_VERSION"] = "2024-08-01-preview"
os.environ["AZURE_DEPLOYMENT_NAME"] = "gpt-4o"


# 创建模型，选择 AZURE 平台和 gpt-4o 模型
model = ModelFactory.create(
    model_platform=ModelPlatformType.AZURE,
    model_type=ModelType.GPT_4O,
    model_config_dict={"temperature": 0.0},
)



In [4]:
os.environ["BOHRIUM_USERNAME"] = "linhang@dp.tech"
os.environ["BOHRIUM_PASSWORD"] = "DP123456"
os.environ["BOHRIUM_PROJECT_ID"] = "21128"

In [5]:
prompt= """
计算1+3，
注意调用工具时使用
- executor:
    type: dispatcher
    machine:
        batch_type: Bohrium
        context_type: Bohrium
        remote_profile:
            input_data:
                image_name: registry.dp.tech/dptech/ubuntu:22.04-py3.10
                job_type: container
                platform: ali
                scass_type: c2_m4_cpu
- storage:
    type: bohrium
之后使用query_job_status查询任务状态
使用get_job_results获取任务结果
"""

In [6]:
import asyncio
from mcp.types import CallToolResult
from camel_sci.toolkits.mcp_toolkit import MCPToolkit, MCPClient
import os
from camel_sci.agents import ChatAgent
from camel_sci.toolkits.human_toolkit import HumanToolkit
human_toolkit = HumanToolkit()
env={
    "BOHRIUM_USERNAME": os.environ.get("BOHRIUM_USERNAME"),
    "BOHRIUM_PASSWORD": os.environ.get("BOHRIUM_PASSWORD"),
    "BOHRIUM_PROJECT_ID": os.environ.get("BOHRIUM_PROJECT_ID"),
}

executor = {
    "type": "dispatcher",
    "machine": {
        "batch_type": "Bohrium",
        "context_type": "Bohrium",
        "remote_profile": {
            "input_data": {
                "image_name": "registry.dp.tech/dptech/ubuntu:22.04-py3.10",
                "job_type": "container",
                "platform": "ali",
                "scass_type": "c2_m4_cpu",
            },
        },
    },
}
storage = {
    "type": "bohrium",
}

async def run_example():
    mcp_client = MCPClient(
        command_or_url="http://127.0.0.1:8000/sse",
        env=env
    )

    await mcp_client.connect()
    mcp_toolkit = MCPToolkit(servers=[mcp_client])
    tools = mcp_toolkit.get_tools()
    for tool in tools:
        print(tool.get_openai_tool_schema())
    try:
        engineer_agent = ChatAgent(
            model=model,
            system_message="你是一个工具人",
            tools=[*tools,*human_toolkit.get_tools()]
        )
        # print(engineer_agent._get_full_tool_schemas())
        response = await engineer_agent.astep(prompt)
        print(response.msgs[0].content)
        print(response.info['tool_calls'])

    except Exception as e:
        print(f"Error during agent execution: {e}")

    finally:
        await mcp_client.disconnect()

if __name__ == "__main__":
    asyncio.run(run_example())

{'type': 'function', 'function': {'name': 'add', 'description': '\n    输入：\n    a:int\n    b:int\n    输出：a+b\n    ', 'parameters': {'type': 'object', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}, 'executor': {'anyOf': [{'type': 'object', 'additionalProperties': False}, {'type': 'null'}], 'title': 'Executor'}, 'storage': {'anyOf': [{'type': 'object', 'additionalProperties': False}, {'type': 'null'}], 'title': 'Storage'}}, 'required': ['a', 'b', 'executor', 'storage'], 'additionalProperties': False}, 'strict': True}}
{'type': 'function', 'function': {'name': 'query_job_status', 'description': '\n    Query status of a calculation job\n    Args:\n        job_id (str): The ID of the calculation job\n    Returns:\n        status (str): One of "Running", "Succeeded" or "Failed"\n    ', 'parameters': {'type': 'object', 'properties': {'job_id': {'title': 'Job Id', 'type': 'string'}, 'executor': {'anyOf': [{'type': 'object', 'additionalProperties'